# **Imports**

In [31]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Load the formated data**

In [104]:
directory = '/content/drive/My Drive/Colab Notebooks/North_Sea/'
exec(open(directory + 'src/inputs_rdn.py').read())

In [90]:
time_years, inp_true_test, chl_stand, NAO_stand, MLD_stand, SST_stand, wind_stand, AMO_stand = var_stand(directory)

# **Method 1**

> ## **Load the model**

In [91]:
params = {}
params['dim_chloro'] = 1
params['dim_phy'] = 5
params['dim_input_vect'] = params['dim_chloro'] + params['dim_phy']
params['dim_input'] = params['dim_input_vect']
params['dim_hidden_1'] = 32
params['dim_hidden_2'] = 28
params['dim_hidden_3'] = 14
params['dim_output'] = params['dim_input']

In [92]:
exec(open(directory + 'src/RNN.py').read())
model_RNN_1 = INT_net(params)
model_RNN_1.load_state_dict(torch.load(directory + 'models/model_1_weights.pth'))

<All keys matched successfully>

> ## **Variables strenght assessment**

In [93]:
## SEE HOW THE ERROR EVOLVES WHEN RANDOMIZING ONE GIVEN PHYSICAL VARIABLE
variables = ['NAO', 'MLD', 'SST', 'wind', 'AMO']
Chl_stand_test = inp_true_test[:,0].reshape(1,-1)[0]
model_RNN_1.double()
for i, var in enumerate(variables):
    print("[%d/%d] : "%(i+1, len(variables)), var)
    MSE_table = []
    for k in range(100):
        inp_test_rdn, inp_true_test_rdn, dt_test = inputs_rdn_1(time_years, chl_stand, NAO_stand, MLD_stand, SST_stand, wind_stand, AMO_stand, randomized_variable = var)
        NN_outputs_test = []
        old_pred = torch.from_numpy(inp_test_rdn[0]).view(1,-1)
        for i in range(len(inp_test_rdn)):
            pred = model_RNN_1(old_pred, torch.from_numpy(dt_test[i]).view(1,-1))
            old_pred = torch.cat((pred[0,0].view(1,-1), torch.from_numpy(inp_true_test_rdn)[i,1:].view(1,-1)), axis = 1)
            NN_outputs_test.append(pred.cpu().detach().numpy()[0][0])

        NMSE = (Chl_stand_test - NN_outputs_test)**2/var_test
        MSE_table.append(np.mean(NMSE))
    print("Mean MSE :", np.mean(MSE_table))
    print("STD  MSE :", np.std(MSE_table))
    print()

[1/5] :  NAO
Mean MSE : 1.2463836029526787
STD  MSE : 0.010316050536275127

[2/5] :  MLD
Mean MSE : 2.2362615740602956
STD  MSE : 0.15464427759407134

[3/5] :  SST
Mean MSE : 3.663465026446289
STD  MSE : 0.2133523254283572

[4/5] :  wind
Mean MSE : 1.338230036965359
STD  MSE : 0.043584932308776185

[5/5] :  AMO
Mean MSE : 1.0073697962520507
STD  MSE : 0.014785658248203003



>> **Variables classification :**
>> 1.   SST
>> 2.   MLD
>> 3.   wind
>> 4.   NAO
>> 5.   AMO



# **Method 2**

> ## **Load the model**

In [94]:
params['dt_horizon'] = 6

In [95]:
model_RNN_2 = INT_net(params)
model_RNN_2.load_state_dict(torch.load(directory + 'models/model_2_weights.pth'))

<All keys matched successfully>

> ## **Variables strenght assessment**

In [96]:
## SEE HOW THE ERROR EVOLVES WHEN RANDOMIZING ONE GIVEN PHYSICAL VARIABLE
variables = ['NAO', 'MLD', 'SST', 'wind', 'AMO']
Chl_stand_test = inp_true_test[:,0].reshape(1,-1)[0]
model_RNN_2.double()
for i, var in enumerate(variables):
    print("[%d/%d] : "%(i+1, len(variables)), var)
    MSE_table = []
    for k in range(100):
        inp_test_rdn, inp_true_test_rdn, dt_test = inputs_rdn_1(time_years, chl_stand, NAO_stand, MLD_stand, SST_stand, wind_stand, AMO_stand, randomized_variable = var)
        NN_outputs_test = []
        old_pred = torch.from_numpy(inp_test_rdn[0]).view(1,-1)
        for i in range(len(inp_test_rdn)):
            pred = model_RNN_2(old_pred, torch.from_numpy(dt_test[i]).view(1,-1))
            old_pred = torch.cat((pred[0,0].view(1,-1), torch.from_numpy(inp_true_test_rdn)[i,1:].view(1,-1)), axis = 1)
            NN_outputs_test.append(pred.cpu().detach().numpy()[0][0])

        NMSE = (Chl_stand_test - NN_outputs_test)**2/var_test
        MSE_table.append(np.mean(NMSE))
    print("Mean MSE :", np.mean(MSE_table))
    print("STD  MSE :", np.std(MSE_table))
    print()

[1/5] :  NAO
Mean MSE : 0.7793703026366796
STD  MSE : 0.008237968237068323

[2/5] :  MLD
Mean MSE : 1.1977046868255399
STD  MSE : 0.0783903877290972

[3/5] :  SST
Mean MSE : 3.8564247251820625
STD  MSE : 0.3068468833160208

[4/5] :  wind
Mean MSE : 1.2465219757842312
STD  MSE : 0.09925900800759528

[5/5] :  AMO
Mean MSE : 0.665491993645905
STD  MSE : 0.010794240030932687



>> **Variables classification :**
>> 1.   SST
>> 2.   wind
>> 3.   MLD
>> 4.   NAO
>> 5.   AMO



# **Method 3**

> ## **Load the model**

In [114]:
params['dt_horizon'] = 4
params['dim_input_vect'] = params['dim_chloro'] + params['dim_phy']
params['dim_input'] = params['dim_input_vect'] * params['dt_horizon'] 
params['dim_hidden_1'] = 32
params['dim_hidden_2'] = 28
params['dim_hidden_3'] = 14
params['dim_output'] = params['dim_input']

In [115]:
model_RNN_3 = INT_net(params)
model_RNN_3.load_state_dict(torch.load(directory + 'models/model_3_weights.pth'))

<All keys matched successfully>

> ## **Variables strenght assessment**

In [116]:
## SEE HOW THE ERROR EVOLVES WHEN RANDOMIZING ONE GIVEN PHYSICAL VARIABLE
variables = ['NAO', 'MLD', 'SST', 'wind', 'AMO']
Chl_stand_test = inp_true_test[:,0].reshape(1,-1)[0]
for i, var in enumerate(variables):
    print("[%d/%d] : "%(i+1, len(variables)), var)
    MSE_table = []
    for k in range(100):
        inp_test_rdn, inp_true_test_rdn, dt_test = inputs_rdn_3(time_years, chl_stand, NAO_stand, MLD_stand, SST_stand, wind_stand, AMO_stand, params['dt_horizon'], randomized_variable = var)
        NN_outputs_test = []
        old_pred = inp_test_rdn[0].view(1,-1)
        for i in range(len(inp_test_rdn)):
            six_prev_months = old_pred
            pred = model_RNN_3(old_pred, torch.ones([1,1]).view(1,-1))
            old_pred = torch.cat((six_prev_months[0, params['dim_input_vect']:].view(1,-1), pred[0,0].view(1,-1)), axis = 1) # drop the oldest month (t-6) of lenght params['dim_input_vect'] = 6, and concat with Chl predicted at month t
            old_pred = torch.cat((old_pred, inp_true_test_rdn[i,1:params['dim_input_vect']].view(1,-1)), axis = 1) # concat with the true physics à month t
            NN_outputs_test.append(pred.cpu().detach().numpy()[0,0])

        NMSE = (Chl_stand_test[params['dt_horizon']:-params['dt_horizon']] - NN_outputs_test)**2/var_test
        MSE_table.append(np.mean(NMSE))
    print("Mean MSE :", np.mean(MSE_table))
    print("STD  MSE :", np.std(MSE_table))
    print()

[1/5] :  NAO
Mean MSE : 1.7049143959436988
STD  MSE : 0.01583642542097043

[2/5] :  MLD
Mean MSE : 1.5784905737856705
STD  MSE : 0.16022638101907563

[3/5] :  SST
Mean MSE : 6.592064109449791
STD  MSE : 2.9326354348838026

[4/5] :  wind
Mean MSE : 2.0349057008975255
STD  MSE : 0.3580008594041561

[5/5] :  AMO
Mean MSE : 1.6979891090791293
STD  MSE : 0.01956283783689452



>> **Variables classification :**
>> 1.   SST
>> 2.   wind
>> 3.   MLD
>> 4.   NAO
>> 5.   AMO

